## Concepts

From one current user find best correlated user

Recommend 5 best rated anime from the best correlated user

Use the best rated anime from current user and recommend 5 more 

Total 10 recommendation is provided 

## Import Library

In [268]:
#pip install pandas
import pandas as pd
import string
import re

## Variables

In [269]:
target_user_id = 75000
target_vote_number = 2000
min_vote_number = 10
numberOfResult = 10

## Loading Dataset
▪ Loading anime.csv and rating.csv into DataFrame

In [270]:
anime_data = pd.read_csv('anime.csv')
#animeDf = animeDf[:5000]
anime_data = pd.DataFrame(anime_data)
anime_data

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211
12290,5543,Under World,Hentai,OVA,1,4.28,183
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175


In [271]:
rating_data = pd.read_csv('rating.csv')
rating_data = rating_data[:1000000]
new_rating_data = pd.read_csv('newRating.csv')
rating_data = pd.DataFrame(rating_data)
rating_data = pd.concat([rating_data, new_rating_data])
rating_data

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1
...,...,...,...
28,75002,136,8
29,75002,136,7
30,75002,136,9
31,75002,136,9


# Data Preprocessing

## Change variable type

In [272]:
anime_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB


In [273]:
anime_data['episodes'] = anime_data['episodes'].replace('Unknown', -1)
anime_data = anime_data.astype({"name":"string","genre":"string","type":"string","episodes":"int64"})

In [274]:
anime_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  string 
 2   genre     12232 non-null  string 
 3   type      12269 non-null  string 
 4   episodes  12294 non-null  int64  
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(3), string(3)
memory usage: 672.5 KB


In [275]:
rating_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000033 entries, 0 to 32
Data columns (total 3 columns):
 #   Column    Non-Null Count    Dtype
---  ------    --------------    -----
 0   user_id   1000033 non-null  int64
 1   anime_id  1000033 non-null  int64
 2   rating    1000033 non-null  int64
dtypes: int64(3)
memory usage: 30.5 MB


## Rename variable

In [276]:
anime_data = anime_data.rename(columns={"rating": "average_rating"})
anime_data.head(3)

,anime_id,name,genre,type,episodes,average_rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262


In [277]:
rating_data = rating_data.rename(columns={"rating": "user_rating"})
rating_data.head(3)

,user_id,anime_id,user_rating
0,1,20,-1
1,1,24,-1
2,1,79,-1


## Check Missing Value

In [278]:
rating_data.isna().sum()

user_id        0
anime_id       0
user_rating    0
dtype: int64

In [279]:
anime_data.isna().sum()

anime_id            0
name                0
genre              62
type               25
episodes            0
average_rating    230
members             0
dtype: int64

## Drop Missing Value

In [280]:
anime_data.dropna(axis=0, inplace= True)
anime_data.isna().sum()

anime_id          0
name              0
genre             0
type              0
episodes          0
average_rating    0
members           0
dtype: int64

## Check Duplicates

In [281]:
#check duplicate anime
duplicated_anime = anime_data[anime_data.duplicated()].shape[0]
print(f'count of duplicated anime: {duplicated_anime}')

count of duplicated anime: 0


In [282]:
#check duplicate rating
duplicated_rating = rating_data[rating_data.duplicated()].shape[0]
print(f'count of duplicated rating: {duplicated_rating}')

count of duplicated rating: 11


In [283]:
rating_data.drop_duplicates(keep='first',inplace=True)
duplicated_rating = rating_data[rating_data.duplicated()].shape[0]
print(f'count of duplicated rating after removing: {duplicated_rating}')

count of duplicated rating after removing: 0


## Merge Dataset by anime_id

In [284]:
raw_merged_data = pd.merge(rating_data, anime_data, on='anime_id', sort = True)
raw_merged_data

,user_id,anime_id,user_rating,name,genre,type,episodes,average_rating,members
0,13,1,-1,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
1,19,1,10,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
2,21,1,9,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
3,23,1,9,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
4,32,1,10,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
...,...,...,...,...,...,...,...,...,...
1000003,6361,34283,7,Nobunaga no Shinobi Episode 0,"Comedy, Historical",Special,1,6.74,937
1000004,7114,34283,6,Nobunaga no Shinobi Episode 0,"Comedy, Historical",Special,1,6.74,937
1000005,2951,34324,-1,Chiryokumaru,Action,ONA,1,5.40,817
1000006,3009,34324,6,Chiryokumaru,Action,ONA,1,5.40,817


## Drop Unnessary Column

In [285]:
#members are not nessesary
#raw_merged_data = raw_merged_data.drop('members', axis=1)
raw_merged_data

,user_id,anime_id,user_rating,name,genre,type,episodes,average_rating,members
0,13,1,-1,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
1,19,1,10,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
2,21,1,9,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
3,23,1,9,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
4,32,1,10,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
...,...,...,...,...,...,...,...,...,...
1000003,6361,34283,7,Nobunaga no Shinobi Episode 0,"Comedy, Historical",Special,1,6.74,937
1000004,7114,34283,6,Nobunaga no Shinobi Episode 0,"Comedy, Historical",Special,1,6.74,937
1000005,2951,34324,-1,Chiryokumaru,Action,ONA,1,5.40,817
1000006,3009,34324,6,Chiryokumaru,Action,ONA,1,5.40,817


## Drop Unnessary Row
Drop the row that less than min_vote_number

In [286]:
#get number of rating for each anime
grouped_data = pd.DataFrame(raw_merged_data.groupby('anime_id')['user_rating'].count())
#get the anime that fulfilled the constaint which is more than min_vote_number
trimmed_data = grouped_data[grouped_data['user_rating'] > min_vote_number]
#trim the data
raw_merged_data = raw_merged_data.loc[raw_merged_data['anime_id'].isin(trimmed_data.index.values.tolist())]
raw_merged_data

,user_id,anime_id,user_rating,name,genre,type,episodes,average_rating,members
0,13,1,-1,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
1,19,1,10,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
2,21,1,9,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
3,23,1,9,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
4,32,1,10,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
...,...,...,...,...,...,...,...,...,...
999996,9213,34240,-1,Shelter,"Music, Sci-Fi",Music,1,8.38,71136
999997,9243,34240,9,Shelter,"Music, Sci-Fi",Music,1,8.38,71136
999998,9278,34240,10,Shelter,"Music, Sci-Fi",Music,1,8.38,71136
999999,9280,34240,7,Shelter,"Music, Sci-Fi",Music,1,8.38,71136


## Clean Text

In [287]:
#before clean text
pd.DataFrame(anime_data.loc[anime_data['anime_id'] == 9969])

,anime_id,name,genre,type,episodes,average_rating,members
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [288]:
def clean_text(text):
    # remove sybmols and other words
    text = re.sub(r'<[^>]*>', '', text)
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'&quot;', '', text)
    text = re.sub(r'.hack//', '', text)
    text = re.sub(r'&#039;', '', text)
    text = re.sub(r'A&#039;s', '', text)
    text = re.sub(r'I&#039;', 'I\'', text)
    text = re.sub(r'&amp;', 'and', text)
    #text = re.sub('[^a-zA-Z0-9\s]','', text)
  
    # remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

In [289]:
raw_merged_data.loc[:, 'name']=raw_merged_data['name'].apply(clean_text)
anime_data.loc[:, 'name'] = anime_data['name'].apply(clean_text)

In [290]:
#after clean text 28977 9969 15417
pd.DataFrame(anime_data.loc[anime_data['anime_id'] == 9969])

,anime_id,name,genre,type,episodes,average_rating,members
4,9969,Gintama,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


# PART 1: User Based Correlation

## Find rating_anime_matrix

In [291]:
rating_anime_matrix = raw_merged_data.pivot_table(index='user_id', columns=['anime_id'], values = 'user_rating', fill_value= -1)
rating_anime_matrix.head()

anime_id,1,5,6,7,8,15,16,17,18,19,...,33372,33421,33524,33558,33569,33606,33964,34085,34103,34240
user_id,,,,,,,,,,,,,,,,,,,,,
1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
5,-1,-1,8,-1,-1,6,-1,6,6,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


## Find anime_rating_matrix

In [292]:
anime_rating_matrix = rating_anime_matrix.transpose()
anime_rating_matrix.head()

user_id,1,2,3,4,5,6,7,8,9,10,...,9352,9353,9354,9355,9356,9357,9358,75000,75001,75002
anime_id,,,,,,,,,,,,,,,,,,,,,
1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,9.0,-1.0,-1.0
5,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
6,-1.0,-1.0,-1.0,-1.0,8.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
7,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
8,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


## Find the information of target_user_id

In [293]:
anime_rating_user = anime_rating_matrix[target_user_id]
pd.DataFrame(anime_rating_user.sort_values(ascending= False)).rename(columns={target_user_id: f"user_id {target_user_id}'s rating"})

,user_id 75000's rating
anime_id,
9253,10.0
32935,10.0
5114,10.0
1,9.0
11061,9.0
...,...
2185,-1.0
2183,-1.0
2182,-1.0


## Find Correlation between target_user_id with all the other users

In [294]:
similar_users = anime_rating_matrix.corrwith(anime_rating_user).dropna()
similar_users

user_id
1       -0.001473
2       -0.000736
3        0.065764
5        0.071269
7        0.056001
           ...   
9356    -0.001647
9357    -0.002949
75000    1.000000
75001    0.256040
75002    0.256040
Length: 8859, dtype: float64

## Format into Data Frame

In [295]:
sorted_similar_users = pd.DataFrame(similar_users, columns=['correlation']).sort_values(ascending= False, by= 'correlation')
sorted_similar_users

,correlation
user_id,
75000,1.000000
4699,0.496943
8581,0.458485
8834,0.421224
4106,0.420796
...,...
5392,-0.013589
4750,-0.013959
721,-0.014257


## Find the best correlated user

In [296]:
#most similar user is the user that at index number 1
most_similar_user = sorted_similar_users.index.values
most_similar_user = most_similar_user[1]
most_similar_user

4699

## Find 5 best rated anime from the user that have highest correlation value

In [314]:
#sort value by rating
target_user = raw_merged_data.loc[raw_merged_data['user_id'] == target_user_id].sort_values(by= 'user_rating', ascending= False)

user_recommendation = raw_merged_data[raw_merged_data['user_id'] == most_similar_user]
user_recommendation = user_recommendation.sort_values(by= 'user_rating', ascending= False)

#Trim the result exist in user's rating
user_recommendation = user_recommendation[~user_recommendation['anime_id'].isin(target_user['anime_id'].tolist())]
user_recommendation = user_recommendation[:numberOfResult]
user_recommendation

,user_id,anime_id,user_rating,name,genre,type,episodes,average_rating,members
377926,4699,2904,10,Code Geass Hangyaku no Lelouch R2,"Action, Drama, Mecha, Military, Sci-Fi, Super ...",TV,25,8.98,572888
283339,4699,1535,10,Death Note,"Mystery, Police, Psychological, Supernatural, ...",TV,37,8.71,1013917
800442,4699,16498,9,Shingeki no Kyojin,"Action, Drama, Fantasy, Shounen, Super Power",TV,25,8.54,896229
259302,4699,1210,8,NHK ni Youkoso,"Comedy, Drama, Psychological, Romance",TV,24,8.40,291228
838058,4699,18679,8,Kill la Kill,"Action, Comedy, School, Super Power",TV,24,8.23,508118
199870,4699,790,7,Ergo Proxy,"Mystery, Psychological, Sci-Fi",TV,23,8.03,265005
816916,4699,17265,7,Log Horizon,"Action, Adventure, Fantasy, Game, Magic, Shounen",TV,25,8.14,387100
6463,4699,19,6,Monster,"Drama, Horror, Mystery, Police, Psychological,...",TV,74,8.72,247562
708980,4699,11757,5,Sword Art Online,"Action, Adventure, Fantasy, Game, Romance",TV,25,7.83,893100
131142,4699,379,5,Heppoko Jikken Animation Excel♥Saga,"Comedy, Parody, Sci-Fi, Shounen",TV,26,7.57,68198


In [298]:
user_recommendation_list = user_recommendation['anime_id'].tolist()
user_recommendation_list

[2904, 1535, 16498, 1210, 18679, 790, 17265, 19, 11757, 379]

# PART 2: Item Based Correlation

## Use the user's highest rated anime to recommend other related anime

In [299]:
target_user_rating = rating_data.loc[rating_data['user_id'] == target_user_id].sort_values(by = 'user_rating', ascending= False)
pd.DataFrame(target_user_rating.head(10))

,user_id,anime_id,user_rating
0,75000,5114,10
1,75000,9253,10
2,75000,32935,10
4,75000,2094,10
3,75000,11061,9
5,75000,1575,9
6,75000,1,9
7,75000,30276,9
13,75000,21,9
8,75000,2001,8


In [300]:
top_rated_anime_id = target_user_rating['anime_id'].tolist()[0]
top_rated_anime_id

5114

In [301]:
ratings = rating_anime_matrix[top_rated_anime_id]
pd.DataFrame(ratings).rename(columns={top_rated_anime_id: f"{top_rated_anime_id}'s rating"})

,5114's rating
user_id,
1,-1
2,-1
3,10
4,-1
5,-1
...,...
9357,-1
9358,-1
75000,10


In [302]:
similar_animes = rating_anime_matrix.corrwith(ratings)
pd.DataFrame(similar_animes, columns=['correlation'])

,correlation
anime_id,
1,0.225968
5,0.142513
6,0.189193
7,0.064520
8,0.024759
...,...
33606,0.044057
33964,0.050053
34085,0.043665


## Identify the most correlated anime

In [303]:
sorted_similar_anime = pd.DataFrame(similar_animes, columns=['correlation']).sort_values(by= 'correlation', ascending= False)
sorted_similar_anime

,correlation
anime_id,
5114,1.000000
121,0.398438
9253,0.383988
10087,0.364962
16498,0.362647
...,...
365,-0.018231
2462,-0.018723
719,-0.018898


## Eliminate the source anime

In [304]:
sorted_similar_anime = sorted_similar_anime[1:]
sorted_similar_anime

,correlation
anime_id,
121,0.398438
9253,0.383988
10087,0.364962
16498,0.362647
2904,0.362011
...,...
365,-0.018231
2462,-0.018723
719,-0.018898


## Ensure the identified anime is popular
Higher number of votes means more popular

In [305]:
#Get number of rating for each anime
rating_votes = pd.DataFrame(raw_merged_data.groupby('anime_id')['user_rating'].count())
rating_votes.rename(columns={'user_rating': 'rating_count'})

,rating_count
anime_id,
1,2116
5,994
6,1487
7,361
8,61
...,...
33606,11
33964,30
34085,11


In [306]:
similar_animes_ratings = sorted_similar_anime.join(rating_votes['user_rating']).sort_values(by = 'correlation', ascending = False)
similar_animes_ratings

,correlation,user_rating
anime_id,,
121,0.398438,3387
9253,0.383988,2308
10087,0.364962,1739
16498,0.362647,3970
2904,0.362011,3282
...,...,...
365,-0.018231,86
2462,-0.018723,20
719,-0.018898,156


## Get the anime that have higher votes and have higher correlation

In [343]:
similar_popular_animes = similar_animes_ratings.loc[similar_animes_ratings['user_rating']>=target_vote_number].dropna()
similar_popular_animes.head()

,correlation,user_rating
anime_id,,
121,0.398438,3387
9253,0.383988,2308
16498,0.362647,3970
2904,0.362011,3282
1575,0.351656,3655


In [351]:
#trim the result exist in user's rating
similar_popular_animes = similar_popular_animes[~similar_popular_animes.index.isin(target_user['anime_id'].tolist())]
similar_popular_animes.head()

,correlation,user_rating
anime_id,,
121,0.398438,3387
16498,0.362647,3970
2904,0.362011,3282
6746,0.341507,2249
3588,0.327292,2518


In [345]:
#make it a list
most_similar_popular_animes = similar_popular_animes[:numberOfResult]
most_similar_popular_animes_list = most_similar_popular_animes.index.to_list()
most_similar_popular_animes_list

[121, 16498, 2904, 6746, 3588, 11757, 9919, 1535, 6547, 19815]

In [309]:
user_recommendation = raw_merged_data[raw_merged_data['user_id'] == most_similar_user]
user_recommendation = user_recommendation.sort_values(by= 'user_rating', ascending= False)
user_recommendation = user_recommendation[:numberOfResult]

# Result Showing

In [346]:
final_recommendation_result = user_recommendation_list.copy()
final_recommendation_result += most_similar_popular_animes_list.copy()
final_recommendation_result

[2904,
 1535,
 16498,
 1210,
 18679,
 790,
 17265,
 19,
 11757,
 379,
 121,
 16498,
 2904,
 6746,
 3588,
 11757,
 9919,
 1535,
 6547,
 19815]

## Show Target User's Rating

In [348]:
target_user = raw_merged_data.loc[raw_merged_data['user_id'] == target_user_id].sort_values(by= 'user_rating', ascending= False)
pd.DataFrame(target_user.head(5))

,user_id,anime_id,user_rating,name,genre,type,episodes,average_rating,members
484283,75000,5114,10,Fullmetal Alchemist Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
617748,75000,9253,10,SteinsGate,"Sci-Fi, Thriller",TV,24,9.17,673572
998128,75000,32935,10,Haikyuu Karasuno Koukou VS Shiratorizawa Gakue...,"Comedy, Drama, School, Shounen, Sports",TV,10,9.15,93351
2115,75000,1,9,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
295039,75000,1575,9,Code Geass Hangyaku no Lelouch,"Action, Mecha, Military, School, Sci-Fi, Super...",TV,25,8.83,715151


## Show Target Anime's information 

In [349]:
target_search_anime = anime_data.loc[anime_data['anime_id'] == top_rated_anime_id]
pd.DataFrame(target_search_anime)

,anime_id,name,genre,type,episodes,average_rating,members
1,5114,Fullmetal Alchemist Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665


## Show Final Recommendation Result

In [350]:
#Drop the duplicated anime
recommended_anime = anime_data.loc[anime_data['anime_id'].isin(final_recommendation_result)]
recommended_anime.drop_duplicates(subset=['anime_id'], keep='first')

pd.DataFrame(recommended_anime)

,anime_id,name,genre,type,episodes,average_rating,members
13,2904,Code Geass Hangyaku no Lelouch R2,"Action, Drama, Mecha, Military, Sci-Fi, Super ...",TV,25,8.98,572888
38,19,Monster,"Drama, Horror, Mystery, Police, Psychological,...",TV,74,8.72,247562
40,1535,Death Note,"Mystery, Police, Psychological, Supernatural, ...",TV,37,8.71,1013917
86,16498,Shingeki no Kyojin,"Action, Drama, Fantasy, Shounen, Super Power",TV,25,8.54,896229
118,19815,No Game No Life,"Adventure, Comedy, Ecchi, Fantasy, Game, Super...",TV,12,8.47,602291
156,1210,NHK ni Youkoso,"Comedy, Drama, Psychological, Romance",TV,24,8.40,291228
159,6547,Angel Beats,"Action, Comedy, Drama, School, Supernatural",TV,13,8.39,717796
166,6746,Durarara,"Action, Mystery, Supernatural",TV,24,8.38,556431
200,121,Fullmetal Alchemist,"Action, Adventure, Comedy, Drama, Fantasy, Mag...",TV,51,8.33,600384
281,18679,Kill la Kill,"Action, Comedy, School, Super Power",TV,24,8.23,508118
